In [7]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp

np.set_printoptions(threshold=np.inf)  # Убирает ограничение на вывод


In [8]:
def factorize_polynomial(coeffs):
    a_n = coeffs[0]

    roots = np.roots(coeffs)

    def format_root(r):
        real_part = f"{r.real:.4g}".rstrip('0').rstrip('.')  # Убираем лишние нули в вещественной части
        imag_part = f"{r.imag:.4g}".rstrip('0').rstrip('.')  # Убираем лишние нули в мнимой части

        if r.imag == 0:
            return f"{real_part}"
        elif r.real == 0:
            return f"{imag_part}j"
        else:
            return f"({real_part} {'+' if r.imag > 0 else '-'} {abs(float(imag_part))}j)"

    factorized_str = f"{a_n:.4g} * " + " * ".join([f"(p - {format_root(r)})" for r in roots])

    return factorized_str, roots


def count_right_left_roots(roots):
    left_count = sum(1 for r in roots if np.real(r) < 0)   # Левые корни
    right_count = sum(1 for r in roots if np.real(r) > 0)  # Правые корни

    return left_count, right_count

In [9]:
def hurwitz_criterion(coefs):
    n = len(coefs) - 1  # степень полинома
    details = "\nРезультат анализа по критерию Гурвица:\n"

    # Необходимое условие: все коэффициенты должны быть положительными.
    if any(c <= 0 for c in coefs):
        details += "Не все коэффициенты положительны. Система неустойчива.\n"
        return False, details

    b = sorted(coefs, key=lambda x: -abs(x))  # Сортировка по убыванию

    # Инициализируем матрицу
    H = np.zeros((n, n))

    # Заполняем матрицу
    for i in range(n):
        for j in range(n):
            idx = 2 * j - i + 1
            if 0 <= idx < len(b):
                H[i, j] = b[idx]
            else:
                H[i, j] = 0

    details += "Матрица Гурвица\n" + str(H) + "\n\n"

    # Вычисляем ведущие определители (главные миноры)
    details += "Мажорные определители\n"
    determinants = []
    for i in range(1, n + 1):
        submatrix = H[:i, :i]
        det = np.linalg.det(submatrix)
        determinants.append(det)
        details += f"Δ{i} = {det:.4f}\n"

    # Если все определители положительны, система устойчива
    is_stable = all(det > 0 for det in determinants)
    if is_stable:
        details += "Все ведущие определители положительны. Система устойчива.\n"
    else:
        details += "Найден отрицательный определитель. Система неустойчива.\n"

    return is_stable, details

In [10]:
p, k, T1, T2, eps = sp.symbols('p k T1 T2 eps')

# Передаточная функция W(p)
W_p = (k * p) / ((T1 * p + 1) * (T2**2 * p**2 + 2 * T2 * eps * p + 1))

# Вычисляем знаменатель
denominator = sp.expand((T1 * p + 1) * (T2**2 * p**2 + 2 * T2 * eps * p + 1))

# Извлекаем коэффициенты
coeffs_symbolic = sp.Poly(denominator, p).all_coeffs()

# Подставляем
# params = {k: 6, T1: 1.2, T2: 0.08, eps: 0.06} # anya
params = {k: 0.5, T1: 6, T2: 4, eps: 0.6}
coeffs_numeric = [float(coeff.subs(params)) for coeff in coeffs_symbolic]

# Вывод коэффициентов
print("Символьные коэффициенты:")
for i, coeff in enumerate(coeffs_symbolic[::-1]):
    print(f"a{i} =", coeff)

print("\nЧисловые коэффициенты:")
for coef in coeffs_numeric:
    print(coef,end="  ")

Символьные коэффициенты:
a0 = 1
a1 = T1 + 2*T2*eps
a2 = 2*T1*T2*eps + T2**2
a3 = T1*T2**2

Числовые коэффициенты:
96.0  44.8  10.8  1.0  

In [11]:
# критерий Гурвица - коэффициенты и определители положительны
hurw_stable, hurw_details = hurwitz_criterion(coeffs_numeric)
print(hurw_details)


Результат анализа по критерию Гурвица:
Матрица Гурвица
[[44.8  1.   0. ]
 [96.  10.8  0. ]
 [ 0.  44.8  1. ]]

Мажорные определители
Δ1 = 44.8000
Δ2 = 387.8400
Δ3 = 387.8400
Все ведущие определители положительны. Система устойчива.



Михайлова

In [12]:
#Вычисляем корни и характерестическое уравнение в виде произведения множителей
factorized_str,roots = factorize_polynomial(coeffs_numeric)

print("\nВыражение:")
print(factorized_str,end="\n\n")

left_roots, right_roots = count_right_left_roots(roots)
print("left_roots  =",left_roots,end="\n")
print("right_roots =",right_roots,end="\n")


Выражение:
96 * (p - (-0.15 + 0.2j)) * (p - (-0.15 - 0.2j)) * (p - -0.1667)

left_roots  = 3
right_roots = 0
